# 5. KPI–OKR Mapping & Implementation
*Aligned with Project Deliverables: KPI–OKR Mapping, Supporting KPIs, and ML-to-Product Metric Mapping*

### 5.1 Strategic Framework: Greener Manufacturing
To ensure our ML project delivers real business value, we map our technical goals directly to the **Mercedes-Benz Ambition 2039** strategy. We are not just predicting numbers; we are optimizing physical factory time to reduce energy usage.

#### **The Objective (O): Optimize Vehicle Testing for Sustainability**
We aim to reduce the time cars spend on test benches, which directly lowers energy consumption (electricity/fuel) and increases factory throughput.

#### **Key Results (KR) & Connected KPIs**
We define success through three distinct layers:

| Key Result (KR) | Supporting KPI (Metric) | Goal | Why it matters |
| :--- | :--- | :--- | :--- |
| **KR 1:** Increase Test Bench Efficiency by 10% | **Mean Absolute Error (MAE)** | Minimize | Shorter tests mean less energy per unit. We need to know (in seconds) how close our prediction is to reality. |
| **KR 2:** Maintain 98% Logistics Reliability | **Under-prediction Rate** | $< 5\%$ | **Critical Risk:** If we predict a test is *short* but it takes *long*, the line stops. This measures the % of dangerous "optimistic" errors. |
| **KR 3:** Validated ML Performance | **$R^2$ Score** | $> 0.55$ | The model must be statistically valid (better than baseline) to be deployed. |

### 5.3 Implementation: The Executive Dashboard
The following code implements this mapping. It transforms raw model predictions into a color-coded dashboard that a Product Manager can read instantly.

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error

def generate_kpi_dashboard(y_true, y_pred, model_name="Model Experiment"):
    """
    Generates a color-coded KPI dashboard connecting ML stats to Business OKRs.
    """
    
    # --- 1. Calculate Core Metrics ---
    
    # Technical Metric: R2 (Variance Explained)
    r2 = r2_score(y_true, y_pred)
    
    # Operational Metric: MAE (Average error in seconds)
    mae = mean_absolute_error(y_true, y_pred)
    
    # Risk Metric: Under-prediction Rate (Line Blockage Risk)
    # Failure Mode: Predicting faster than reality (Residual < 0)
    residuals = y_pred - y_true
    under_prediction_count = np.sum(residuals < 0)
    under_prediction_rate = under_prediction_count / len(y_true)
    
    # Business Metric: CO2 Impact (Hypothetical: 0.05g CO2 per second of error)
    co2_factor_per_sec = 0.05 
    total_error_seconds = np.sum(np.abs(residuals))
    estimated_wasted_emissions = total_error_seconds * co2_factor_per_sec

    # --- 2. Define Targets (The "OKR" part) ---
    r2_target = 0.55
    risk_target = 0.05 # Max 5% failure allowed

    # --- 3. Build the Data Table ---
    data = {
        "Metric Category": [
            "Technical Viability (KR3)", 
            "Operational Efficiency (KR1)", 
            "Logistics Risk (KR2)", 
            "Sustainability Impact (Obj)"
        ],
        "KPI Name": [
            "R2 Score", 
            "Mean Absolute Error (MAE)", 
            "Under-prediction Rate (Line Blockage)", 
            "Est. Wasted Emissions"
        ],
        "Value": [
            r2, 
            mae, 
            under_prediction_rate, 
            estimated_wasted_emissions
        ],
        "Target": [
            f"> {r2_target}", 
            "Minimize", 
            f"< {risk_target:.0%}", 
            "Minimize"
        ],
        "Status": [
            "PASS" if r2 > r2_target else "FAIL",
            "TRACK",
            "PASS" if under_prediction_rate < risk_target else "CRITICAL",
            "TRACK"
        ]
    }
    
    df = pd.DataFrame(data)

    # --- 4. Styling for Visual Output ---
    def color_status(val):
        color = 'white'
        if val == 'PASS':
            color = '#8FBC8F' # Dark Sea Green
        elif val == 'FAIL' or val == 'CRITICAL':
            color = '#FF7F7F' # Light Red
        elif val == 'TRACK':
            color = '#F0E68C' # Khaki
        return f'background-color: {color}; color: black; font-weight: bold'

    # FIXED: Replaced .applymap with .map
    styled_df = df.style.map(color_status, subset=['Status'])\
        .format({'Value': "{:.4f}"})\
        .set_caption(f"KPI-OKR Dashboard: {model_name}")\
        .set_properties(**{'text-align': 'left'})

    return styled_df

# --- EXECUTION BLOCK ---
print("Running KPI Dashboard simulation...")

# Simulate "Ground Truth" (Actual times)
dummy_y_true = np.array([100, 78, 90, 110, 85, 92, 105, 88])

# Simulate "Model Predictions" 
dummy_y_pred = np.array([98, 80, 85, 108, 95, 90, 102, 80]) 

# Generate and DISPLAY the dashboard
dashboard = generate_kpi_dashboard(dummy_y_true, dummy_y_pred, "XGBoost_Baseline_v1")
display(dashboard)

Running KPI Dashboard simulation...


,Metric Category,KPI Name,Value,Target,Status
0,Technical Viability (KR3),R2 Score,0.7338,> 0.55,PASS
1,Operational Efficiency (KR1),Mean Absolute Error (MAE),4.2500,Minimize,TRACK
2,Logistics Risk (KR2),Under-prediction Rate (Line Blockage),0.7500,< 5%,CRITICAL
3,Sustainability Impact (Obj),Est. Wasted Emissions,1.7000,Minimize,TRACK
